# FinBERT for Sentinment Analysis

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
from bs4 import BeautifulSoup
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
from official.nlp import optimization
import tqdm
import pandas as pd

tf.get_logger().setLevel('ERROR')

C:\Users\MB-PC\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
# create a tokenizer object
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# fetch the pretrained model 
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [3]:
# A headline to be used as input 
headline = "Bitcoin is on the same price for the day"

# Pre-process input phrase
input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
# Run inference on the tokenized phrase
output = model(**input)

# Pass model output logits through a softmax layer.
sentim_scores = torch.nn.functional.softmax(output.logits, dim=-1)

In [4]:
sentim_scores

tensor([[0.0371, 0.0226, 0.9402]], grad_fn=<SoftmaxBackward0>)

In [5]:
data = pd.read_csv("C:/Users/MB-PC/ML/Cardano_reddit_extraction_101.csv")

In [6]:
data = data.rename(columns={'Body':'headline'})
#data["headline"] = data["Content"]
#data = data.drop(["Content","Unnamed: 0"],axis=1)

In [7]:
data = data.dropna()

In [8]:
data = data.drop(["Unnamed: 0"],axis=1)
data
#data['headline'] = data['headline'].astype(str)

,index,headline,Date
0,0,"They have lots of funding and marketing, but d...",2023-05-13 10:11:06
1,1,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-05-13 05:30:10
2,2,Just have a look at Cardano's development comp...,2023-05-13 01:12:59
3,3,Really awesome time having a chance to chat wi...,2023-05-13 01:03:11
4,4,"Me: \n\nReferring to your data, what exactly d...",2023-05-12 21:23:28
...,...,...,...
532,955,&#x200B;\n\nhttps://preview.redd.it/fpmk2lxpz6...,2023-02-24 20:05:11
533,956,"Cardano, a blockchain platform, has a strong c...",2023-02-24 14:02:36
534,959,Hey Everyone!\n\nLet’s go….\n\nNewly Covered T...,2023-02-24 07:43:53
535,961,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-02-24 06:30:09


In [9]:
if 'headline' in data:
    print('true')
else: print('false')     

true


In [15]:

part_data = data.head(4)
#data.dtypes

In [11]:
def sentim_analyzer(df, tokenizer, model):
    for i in range (0, len(df)) :
        try:
            headline = df.loc[i, 'headline']
            
        except:
            return print(' \'headline\' column might be missing from dataframe')
        # Pre-process input phrase
        input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
        # Estimate output
        output = model(**input)
        # Pass model output logits through a softmax layer.
        predictions = torch.nn.functional.softmax(output.logits, dim=-1)
        df.loc[i, 'Positive'] = predictions[0][0].tolist()
        df.loc[i, 'Negative'] = predictions[0][1].tolist()
        df.loc[i, 'Neutral']  = predictions[0][2].tolist()
    # rearrange column order
    try:
        df = df[['date',  'headline', 'Positive', 'Negative', 'Neutral']]
    except:
        pass
    return df

In [19]:
sentim_data = sentim_analyzer(data, tokenizer, model)

In [20]:
sentim_data

,headline,Date,Positive,Negative,Neutral
0,"They have lots of funding and marketing, but d...",2023-05-13 10:11:06,0.089965,0.037385,0.872649
1,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-05-13 05:30:10,0.019929,0.085898,0.894173
2,Just have a look at Cardano's development comp...,2023-05-13 01:12:59,0.069088,0.015764,0.915147
3,Really awesome time having a chance to chat wi...,2023-05-13 01:03:11,0.231345,0.010165,0.758489
4,"Me: \n\nReferring to your data, what exactly d...",2023-05-12 21:23:28,0.039080,0.016425,0.944495
...,...,...,...,...,...
532,&#x200B;\n\nhttps://preview.redd.it/fpmk2lxpz6...,2023-02-24 20:05:11,0.125506,0.012386,0.862108
533,"Cardano, a blockchain platform, has a strong c...",2023-02-24 14:02:36,0.147318,0.010207,0.842475
534,Hey Everyone!\n\nLet’s go….\n\nNewly Covered T...,2023-02-24 07:43:53,0.018677,0.510275,0.471048
535,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-02-24 06:30:09,0.019929,0.085898,0.894173


In [ ]:
# Positive + Negative + Neutral = [0,5]
# Score = Positive Magnitude + (0.5 * Neutral Magnitude)) / (Positive Magnitude + Neutral Magnitude + Negative Magnitude) * 5


#sentim_data["Score"] = (sentim_data["Positive"] + 0.5*sentim_data["Neutral"])*5 / (sentim_data["Positive"]+ sentim_data["Neutral"]+ sentim_data["Negative"])

In [21]:
# sentiment Between 0 and 1

sentim_data["Score"] = (sentim_data["Positive"] + 0.5*sentim_data["Neutral"]) / (sentim_data["Positive"]+ sentim_data["Neutral"]+ sentim_data["Negative"]) 


In [22]:
sentim_data.head(4)

,headline,Date,Positive,Negative,Neutral,Score
0,"They have lots of funding and marketing, but d...",2023-05-13 10:11:06,0.089965,0.037385,0.872649,0.526290
1,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-05-13 05:30:10,0.019929,0.085898,0.894173,0.467015
2,Just have a look at Cardano's development comp...,2023-05-13 01:12:59,0.069088,0.015764,0.915147,0.526662
3,Really awesome time having a chance to chat wi...,2023-05-13 01:03:11,0.231345,0.010165,0.758489,0.610590


In [23]:
sentim_data.to_csv("sentiment_with_results_cardano101_unknown.csv")

In [55]:
final_score = pd.read_csv("sentiment_with_results_cardano101_unknown.csv")

# Assuming your DataFrame is named 'final_score' and the timestamp column is the only column

final_score['Date'] = pd.to_datetime(final_score['Date']).dt.date
final_score = final_score.groupby('Date')['Score'].mean()



In [57]:
final_score.to_csv("final_score_cardano101_unknown.csv")
#sentim_data = pd.read_csv('sentiment_with_results_etherum.csv')

In [54]:
#grouped_data = df.groupby('Date')['Score'].mean()

final_score = final_score.groupby('Date')['Score'].mean()

KeyError: 'Column not found: Score'

In [31]:
final_score.to_csv("final_sentiment_score_cardano_101.csv")

In [49]:
print(final_score.columns)

Index(['Unnamed: 0', 'headline', 'Date', 'Positive', 'Negative', 'Neutral',
       'Score'],
      dtype='object')


In [39]:
final_score.rename(columns={'Date':'date'}, inplace = True)

In [53]:
final_score

Date
2023-02-23 21:35:30    0.580403
2023-02-24 06:30:09    0.467015
2023-02-24 07:43:53    0.254201
2023-02-24 14:02:36    0.568556
2023-02-24 20:05:11    0.556560
                         ...   
2023-05-12 21:23:28    0.511328
2023-05-13 01:03:11    0.610590
2023-05-13 01:12:59    0.526662
2023-05-13 05:30:10    0.467015
2023-05-13 10:11:06    0.526290
Name: Score, Length: 537, dtype: float64

In [50]:

# Assuming your DataFrame is named 'df' and the datetime column is named 'Date'


In [51]:
final_score

,Unnamed: 0,headline,Date,Positive,Negative,Neutral,Score
0,0,"They have lots of funding and marketing, but d...",2023-05-13,0.089965,0.037385,0.872649,0.526290
1,1,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-05-13,0.019929,0.085898,0.894173,0.467015
2,2,Just have a look at Cardano's development comp...,2023-05-13,0.069088,0.015764,0.915147,0.526662
3,3,Really awesome time having a chance to chat wi...,2023-05-13,0.231345,0.010165,0.758489,0.610590
4,4,"Me: \n\nReferring to your data, what exactly d...",2023-05-12,0.039080,0.016425,0.944495,0.511328
...,...,...,...,...,...,...,...
532,532,&#x200B;\n\nhttps://preview.redd.it/fpmk2lxpz6...,2023-02-24,0.125506,0.012386,0.862108,0.556560
533,533,"Cardano, a blockchain platform, has a strong c...",2023-02-24,0.147318,0.010207,0.842475,0.568556
534,534,Hey Everyone!\n\nLet’s go….\n\nNewly Covered T...,2023-02-24,0.018677,0.510275,0.471048,0.254201
535,535,"Hello everyone,\n\nWelcome to the Cardano Dail...",2023-02-24,0.019929,0.085898,0.894173,0.467015
